In [1]:
import time
import pandas as pd
import re

import datetime as dt
from dateutil.relativedelta import relativedelta

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException, TimeoutException

In [2]:
driver = webdriver.Chrome('chromedriver')
data_dic = {
     'title' : []
    ,'content' : []
    ,'date' : []
    ,'url' : []
}

In [3]:
yyyymm_1 = '200801'
yyyymm_2 = '201212'
# yyyymm_1 = '201301'
# yyyymm_2 = '201812'

MONTH = relativedelta(months=+1)

fmt = '%Y%m'
date_1 = dt.datetime.strptime(yyyymm_1, fmt).date()
date_2 = dt.datetime.strptime(yyyymm_2, fmt).date()

yyyymm = []
while date_1 <= date_2:
    yyyymm.append(date_1.strftime('%Y')+date_1.strftime('%m'))
    date_1 += MONTH

In [ ]:
def init(start_date, end_date):
    url = "https://search.naver.com/search.naver?where=news&query=&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={}&de={}&docid=&nso=so%3Ar%2Cp%3Afrom{}to{}%2Ca%3Aall&mynews=0&refresh_start=0&related=0".format(start_date, end_date, start_date.replace(".",""), end_date.replace(".",""))
    driver.get(url)
    
    # 검색옵션 클릭
    # driver.find_element_by_css_selector('#_search_option_btn').click()

    # 최신순 선택
    driver.find_element_by_css_selector('#snb > div > ul > li:nth-child(3) > a').click()
    driver.find_element_by_css_selector('#snb > div > ul > li.menu.selected > div > ul > li:nth-child(2) > a').click()

    # 언론사 선택
    driver.find_element_by_css_selector('#news_popup > a').click()
    # 연합뉴스 체크
    driver.find_element_by_css_selector('#ca_1001').click()
    # 확인
    driver.find_element_by_css_selector('#_nx_option_media > div.con_bx > div.view_btn > button.impact._submit_btn').click()

    # 뉴스 키워드 입력
    keyword = '날씨'
    # 검색
    driver.find_element_by_css_selector('#news_inner_search_query').send_keys(keyword)
    driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div > div.news_srch > span > button').click()
    # 최신순 정렬
    driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.section_head > div.news_option > ul > li.select').click()

    # 전체 번호 세팅
    total_page = driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.section_head > div.title_desc.all_my > span').text
    total_page = total_page.split('/')
    total_page = total_page[1]
    total_page = total_page.replace(" ", "")
    total_page = total_page.replace(",", "")
    total_page = total_page.replace("건", "")
    total_page = int(total_page)

    # 현재 번호
    current_page = driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.paging > strong').text
    current_page = int(current_page)
    return total_page, current_page

In [ ]:
# 다 끝났으면 다음 페이지로 이동하는 모듈
def moveNextPage():
    global current_page
    current_page = current_page + 1
    # 페이징 태그 가져오기
    paging_elements = driver.find_elements_by_css_selector(".paging > a")
    for paging_num in paging_elements:
        if(paging_num.text == str(current_page)):
            paging_num.click()
            break

In [ ]:
def getContent():
    content = ''
    date = ''
    url = ''
    try:
        content = driver.find_element_by_css_selector('#articleBodyContents').text
        date = "".join(re.findall('\d+',driver.find_element_by_css_selector('#main_content > div.article_header > div.article_info > div > span:nth-child(1)').text[0:10]))
        url = driver.current_url
    except:
        print(driver.current_url,"오류 발생")
        content = "오류"
        url = driver.current_url
    finally:
        return content, date, url

In [ ]:
for i in range(int(len(yyyymm) / 4)):
    ds = yyyymm[i*4][0:4]  +'.'+yyyymm[i*4][4:6]+'.01'
    de = yyyymm[i*4+3][0:4]+'.'+yyyymm[i*4+3][4:6]+'.31'
    [total_page, current_page] = init(ds, de)
    while(True):
        page_content = driver.find_elements_by_css_selector('.type01 > li')
        for content in page_content:
            # 제목 담기
            data_dic['title'].append(content.find_element_by_css_selector('._sp_each_title').text)

            # 링크 클릭(네이버 뉴스로 대체)
            content.find_element_by_css_selector('._sp_each_url').click()
            # 최근 열린 탭으로 전환
            driver.switch_to.window(driver.window_handles[-1])

            ### 새로운 탭에서 작업 ###
            [content, date, url] = getContent()
            data_dic['content'].append(content)
            data_dic['date'].append(date)
            data_dic['url'].append(url)

            # 현재 탭 닫기
            driver.close()
            # 맨 처음 탭으로 변경(0번 탭)
            driver.switch_to.window(driver.window_handles[0])
        if(current_page * 10 >= total_page | current_page * 10 >= 4000):
            print(current_page*10,'크롤링 끝!')
            break
        else:
            moveNextPage()

https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0002062649 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0002062520 오류 발생
https://entertain.naver.com/read?oid=001&aid=0002062219 오류 발생
https://entertain.naver.com/read?oid=001&aid=0002061409 오류 발생
https://entertain.naver.com/read?oid=001&aid=0002061146 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0002059985 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0002058272 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0002058270 오류 발생
https://entertain.naver.com/read?oid=001&aid=0002058090 오류 발생
https://entertain.naver.com/read?oid=001&aid=0002057467 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0002056451 오류 발생
https://entertain.naver.com/read?oid=001&aid=0002055339 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0002053307 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&a

https://entertain.naver.com/read?oid=001&aid=0001938745 오류 발생
https://entertain.naver.com/read?oid=001&aid=0001938602 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0001937355 오류 발생
https://entertain.naver.com/read?oid=001&aid=0001936138 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0001935774 오류 발생
https://entertain.naver.com/read?oid=001&aid=0001935510 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0001935042 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0001931317 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0001930437 오류 발생
https://sports.news.naver.com/general/news/read.nhn?oid=001&aid=0001928298 오류 발생
https://entertain.naver.com/read?oid=001&aid=0001927043 오류 발생
https://entertain.naver.com/read?oid=001&aid=0001927044 오류 발생
https://entertain.naver.com/read?oid=001&aid=0001927045 오류 발생
https://entertain.naver.com/read?oid=001&aid=0001926723 오류 발생
https://entertain.

Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\history.py", line 578, in end_session
    len(self.input_hist_parsed)-1, self.session_number))
sqlite3.OperationalError: database is locked


In [ ]:
df = pd.DataFrame.from_dict(data_dic)
df.to_csv(yyyymm_1[0:4]+yyyymm_2[0:4]+'날씨.csv', index=False, encoding='utf-8-sig')
# df.to_csv('test.csv', index=False, encoding='utf-8-sig')